<a href="https://colab.research.google.com/github/AminaMOUDJAR/T5_fine_tuned/blob/main/arat5_MSA_small_arabic2english_sum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate datasets arabert

In [ ]:
while True: pass

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!wandb login "ec4fd36593b5e8306001ca236dd6efc6424b46dc"

In [ ]:
import datasets
import transformers
import pandas as pd
from datasets import Dataset
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
train_set = pd.read_excel("../datasets/dataset_XL_sum_v1.0_train_ar_en_recentTrain.xlsx")
test_set = pd.read_excel("../datasets/dataset_XL_sum_v1.0_test_ar_en_recentTest.xlsx")

In [ ]:
train_set['Ar_Summary'] = train_set['Ar_Summary'].replace(r'\n', '', regex = True)
train_set['En_Summary'] = train_set['En_Summary'].replace(r'\n', '', regex = True)

test_set['Ar_Summary'] = test_set['Ar_Summary'].replace(r'\n', '', regex = True)
test_set['En_Summary'] = test_set['En_Summary'].replace(r'\n', '', regex = True)

#train_set = train_set.drop(columns = ['Unnamed: 0'])
#test_set = test_set.drop(columns = ['Unnamed: 0'])
#train_set = train_set.drop(columns = ['Unnamed: 0.1'])
#test_set = test_set.drop(columns = ['Unnamed: 0.1'])

print(train_set.shape)
print(test_set.shape)
# train_set = train_set.drop(range(97,9997))
# test_set = test_set.drop(range(10,2000))

train_set = train_set.dropna()
test_set = test_set.dropna()

print(train_set.shape)
print(test_set.shape)
for col in train_set.columns:
    print(col)
for col in test_set.columns:
    print(col)

NameError: name 'train_set' is not defined

In [ ]:
# Average length of documents and summaries
avg_matn_length = train_set["Ar_Document"].str.split().str.len().mean()
avg_bab_length = train_set["Ar_Summary"].str.split().str.len().mean()

# max input and output length according to the average lengths
max_input_length = int(avg_matn_length)
max_target_length = int(avg_bab_length)

print("Document average length:", max_input_length)
print("Summary average length:", max_target_length)

Document average length: 423
Summary average length: 25


In [ ]:
train_ds = Dataset.from_pandas(train_set)
test_ds = Dataset.from_pandas(test_set)

In [ ]:
train_dataset, validation_dataset= train_ds.train_test_split(test_size=0.10).values()
data_all_splits = datasets.DatasetDict({"train":train_dataset,"test":test_ds, "val":validation_dataset})

In [ ]:
!pip install arabert

In [ ]:
# from arabert.preprocess import ArabertPreprocessor
# preprocessor = ArabertPreprocessor(model_name="aubmindlab/bert-base-arabertv02")

# for i in range(len(train_set)):
#     train_set.loc[i, "Ar_Document"] = preprocessor.preprocess(str(train_set.loc[i, "Ar_Document"]))
#     train_set.loc[i, "Ar_Summary"] = preprocessor.preprocess(str(train_set.loc[i, "Ar_Summary"]))

# for i in range(len(test_set)):
#     test_set.loc[i, "Ar_Document"] = preprocessor.preprocess(str(test_set.loc[i, "Ar_Document"]))
#     test_set.loc[i, "Ar_Summary"] = preprocessor.preprocess(str(test_set.loc[i, "Ar_Summary"]))

In [ ]:
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.0 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [ ]:
from transformers import AutoTokenizer, BertTokenizer, AutoModelForSeq2SeqLM, pipeline
from arabert.preprocess import ArabertPreprocessor
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_name="UBC-NLP/AraT5-msa-small"
#model_name="malmarjeh/mbert2mbert-arabic-text-summarization"
#preprocessor = ArabertPreprocessor(model_name=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/100 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

In [ ]:
max_input_length = 64
max_target_length = 64

source_lang = "ar"
target_lang = "en"

prefix = "translate Arabic to English: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Ar_Summary"]]
    targets = [doc for doc in examples["En_Summary"]]
    #inputs=preprocessor.preprocess(inputs)
    model_inputs = tokenizer(inputs,
                             text_target=targets,
                             max_length=max_input_length,
                             truncation=True,
                             padding='max_length',
                             return_attention_mask=True,
                             add_special_tokens=True)
    # Setup the tokenizer for targets
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(
#             examples["En_Summary"],
#             max_length=max_target_length,
#             truncation=True,
#             padding='max_length',
#             return_attention_mask=True,
#             add_special_tokens=True
#         )

#     model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# def preprocess_function(examples):

#     inputs = [prefix + example[source_lang] for example in examples["translation"]]

#     targets = [example[target_lang] for example in examples["translation"]]

#     model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)

#     return model_inputs

In [ ]:
tokenized_datasets = data_all_splits.map(preprocess_function, batched=True)

Map:   0%|          | 0/30190 [00:00<?, ? examples/s]

Map:   0%|          | 0/8387 [00:00<?, ? examples/s]

Map:   0%|          | 0/3355 [00:00<?, ? examples/s]

In [ ]:
batch_size =5
model_name = model_name
args = Seq2SeqTrainingArguments(
    f"art5_ar_en_sum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=5,

    #predict_with_generate=True,
    #fp16=True,
    #push_to_hub=True,
    #load_best_model_at_end=True, #need to change strategies
    overwrite_output_dir=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
import os
import gc
import torch
torch.cuda.empty_cache()
gc.collect()
os.environ["WANDB_DISABLED"] = "true"
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
trainer.train()

Epoch,Training Loss,Validation Loss
1,4.187500,3.921758
2,3.858600,3.604123
3,3.688600,3.380750
4,3.606700,3.230503
5,3.504200,3.182277


TrainOutput(global_step=30190, training_loss=4.253729969794168, metrics={'train_runtime': 3514.2996, 'train_samples_per_second': 42.953, 'train_steps_per_second': 8.591, 'total_flos': 3283749509529600.0, 'train_loss': 4.253729969794168, 'epoch': 5.0})

In [ ]:
trainer.save_model("ara_t5_msa_Ara_Eng_Sum_Trained")

In [ ]:
from arabert.preprocess import ArabertPreprocessor
#preprocessor = ArabertPreprocessor(model_name="aubmindlab/bert-base-arabertv02")


In [ ]:
from transformers import pipeline
model_name="ara_t5_msa_Ara_Eng_Sum_Trained"#Use the model already saved after training
preprocessor = ArabertPreprocessor(model_name="")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
pipeline = pipeline("translation",model=model,tokenizer=tokenizer,max_length=64,truncation=True)


In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Convert the tokenizer to the fast tokenizers
tokenizer.add_special_tokens({
    'additional_special_tokens': ['[CUSTOM_TOKEN]'],
})
tokenizer.add_prefix_space = True

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def translate(example):
    example['Predicted_En_Summary'] =pipeline(example['Predicted_Ar_Summary'],
            pad_token_id=tokenizer.eos_token_id,
            num_beams=3,#num_samples=3
            #num_return_sequences=1,
            repetition_penalty=3.0
            )[0]['translation_text']
    return example

In [ ]:
torch.cuda.empty_cache()
#test_ds = test_ds.drop(test_ds.index[[741, 742]])
gc.collect()

79

In [ ]:
test_ds = test_ds.map(translate ,batched = False)

Map:   0%|          | 0/8387 [00:00<?, ? examples/s]

KeyError: 'text'

In [ ]:
print(translate)

<function translate at 0x7a61d67e1360>


In [ ]:
example = test_ds[0]
print('text' in example)

False


In [ ]:
test_ds = test_ds.map(lambda example: {'Predicted_En_Summary': pipeline(example['Ar_Summary'],
              pad_token_id=tokenizer.eos_token_id,
              num_beams=3,#num_samples=3
              #num_return_sequences=1,
              )},
              batched = False)

Map:   0%|          | 0/8387 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
print(test_ds)

Dataset({
    features: ['Ar_Document', 'Ar_Summary', 'En_Summary'],
    num_rows: 8387
})


In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
pipeline = pipeline("translation", model="Helsinki-NLP/opus-mt-ar-en", tokenizer=tokenizer)

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
def translate(example):
    example['Predicted_En_Summary'] = pipeline(example['text'],
             pad_token_id=tokenizer.eos_token_id,
             num_beams=3,#num_samples=3
             #num_return_sequences=1,
            )
    return example

In [ ]:
test_set=test_ds.to_pandas()
test_set

In [ ]:
test_set.to_excel('ara_t5_msa_Ara_Eng_Sum_Test.xlsx',index=False)#Save the results into a excel file
print(test_set)

In [ ]:
!pip install evaluate
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=bd22a8c9835918c666c70c178e0d4064373531e16efbdb62dd620f8f6c3fea00
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import evaluate

In [ ]:
rouge_score = evaluate.load("rouge")

In [ ]:
df = pd.read_excel("dataset_XL_sum_v1.0_test_ar_en_recentTest.xlsx")
df.Ar_Summary = df.Ar_Summary.replace(r'\s+', ' ', regex=True)
df.Predicted_Ar_Summary = df.Predicted_Ar_Summary.replace(r'\s+', ' ', regex=True)
print(df.head())

AttributeError: 'DataFrame' object has no attribute 'Predicted_Ar_Summary'

In [ ]:
def evaluate_baseline(dataset, metric):
    return rouge_score.compute(predictions=df["Predicted_En_Summary"], references=df["En_Summary"], tokenizer=lambda x: x.split())

In [ ]:
score = evaluate_baseline(test_set, rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn] * 100, 2)) for rn in rouge_names)
rouge_dict

KeyError: 'Predicted_En_Summary'